In [ ]:
import pandas as pd
import numpy as np
import quandl, math
df = quandl.get("WIKI/GOOGL")
print(df.head())

In [ ]:
df = df[['Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume']]

In [ ]:
df['HL_PCT'] = (df['Adj. High'] - df['Adj. Low']) / df['Adj. Close']
df['PCT_change'] = (df['Adj. Close'] - df['Adj. Open']) / df['Adj. Open'] * 100.0
df = df[['Adj. Close', 'HL_PCT', 'PCT_change', 'Adj. Volume']]
print(df.head())

In [ ]:
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

forecast_col = 'Adj. Close' 
df.fillna(value=-99999, inplace=True)
forecast_out = int(math.ceil(0.01 * len(df)))
df['label'] = df[forecast_col].shift(-forecast_out)

In [ ]:
df.dropna(inplace=True)

In [ ]:
X = np.array(df.drop(['label'], 1))
df.dropna(inplace=True)
y = np.array(df['label'])

X = preprocessing.scale(X)
y = np.array(df['label'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
clf = svm.SVR()

clf.fit(X_train, y_train)
confidence = clf.score(X_test, y_test)

print(confidence)

In [ ]:
clf = LinearRegression()
clf.fit(X_train, y_train)
confidence = clf.score(X_test, y_test)
print(confidence)

In [ ]:
clf = LinearRegression(n_jobs=-1)

In [ ]:
for k in ['linear','poly','rbf','sigmoid']:
    clf = svm.SVR(kernel=k)
    clf.fit(X_train, y_train)
    confidence = clf.score(X_test, y_test)
    print(k,confidence)

In [ ]:
X = np.array(df.drop(['label'], 1))
X = preprocessing.scale(X)
X_lately = X[-forecast_out:]
X = X[:-forecast_out]

In [ ]:
df.dropna(inplace=True)

In [ ]:
y = np.array(df['label'])

In [ ]:
df.dropna(inplace=True)

y = np.array(df['label'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

clf = LinearRegression(n_jobs=-1)
clf.fit(X_train, y_train)
confidence = clf.score(X_test, y_test)

print(confidence)

In [ ]:
forecast_set = clf.predict(X_lately)
print(forecast_set, confidence, forecast_out)

In [ ]:
import datetime
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')

In [ ]:
df['Forecast'] = np.nan

In [ ]:
last_date = df.iloc[-1].name
last_unix = last_date.timestamp()
one_day = 86400 #24×60×60
next_unix = last_unix + one_day

In [ ]:
for i in forecast_set:
    next_date = datetime.datetime.fromtimestamp(next_unix)
    next_unix += 86400
    df.loc[next_date] = [np.nan for _ in range(len(df.columns)-1)]+[i]


In [ ]:
df['Adj. Close'].plot()
df['Forecast'].plot()
plt.legend(loc=4)
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()